<a href="https://colab.research.google.com/github/Hq030302/Sentiment-Analysis/blob/labelling-pake-netral/labell_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=1ff1782ebc9f3ca393be097a214adf8eb8aae781b3a255ce272b291468411b0e
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling ch

In [ ]:
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
data = pd.read_csv('filtered.csv')
data = data.dropna()
print(data.head())

   Unnamed: 0                      created_at  \
0           0  Thu Nov 02 07:06:28 +0000 2023   
1           1  Thu Nov 02 06:59:57 +0000 2023   
2           2  Thu Nov 02 06:59:31 +0000 2023   
3           3  Thu Nov 02 06:59:21 +0000 2023   
4           4  Thu Nov 02 06:50:44 +0000 2023   

                                           full_text  quote_count  \
0  Legislatif PKS Desak BPK Audit Proyek LRT Jabo...            0   
1  @pakmul63 @PKSejahtera @FPKSDPRRI Udah dapet g...            0   
2  @shortsidestory lrt jabodebek kan gak pke masi...            0   
3  LRT Jabodebek kembali ke headway 15 menit seka...            1   
4  Oh jadi ini ya alasannya LRT Jabodebek rolling...            0   

   reply_count  retweet_count  favorite_count lang         username  
0            0              1               3   in        tvOneNews  
1            0              0               0   in  frizkiramadhani  
2            0              0               0   in        aliyaff28  
3         

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
from googletrans.client import Translator
translator = Translator()

In [ ]:
def clean_tweets(tweet):
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [ ]:
import re
data['clean'] = data.apply(lambda x: clean_tweets(x['full_text']), axis=1)
print(data['clean'])

0      Legislatif PKS Desak BPK Audit Proyek LRT Jabo...
1      Udah dapet gambaran majunya industri kita sete...
2      lrt jabodebek kan gak pke masinis ya tapi yg j...
3      LRT Jabodebek kembali ke headway 15 menit seka...
4      Oh jadi ini ya alasannya LRT Jabodebek rolling...
                             ...                        
104    Commision nada chime yg original mahal ya buat...
105    Kemenhub Buka Suara Soal Polemik LRT Jabodebek...
106      Kemenhub Janji Percepat Perawatan LRT Jabodebek
107                             telat dikit gangaruh ygy
108    WKWKWKWKW YG 7D SANNN SELALU KOSONG MELOMPONG ...
Name: clean, Length: 109, dtype: object


In [ ]:
data['label'] = data['full_text'].str.encode('ascii','ignore').apply(translator.translate, dest='en')
data['label'] = data['label'].apply(getattr, args=('text',))
data['label'] = data.apply(lambda x: clean_tweets(x['label']), axis=1)

In [ ]:
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["label"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["label"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["label"]]
data['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in data["label"]]
data.head()

,Unnamed: 0,created_at,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,username,clean,label,Positive,Negative,Neutral,Compound
0,0,Thu Nov 02 07:06:28 +0000 2023,Legislatif PKS Desak BPK Audit Proyek LRT Jabo...,0,0,1,3,in,tvOneNews,Legislatif PKS Desak BPK Audit Proyek LRT Jabo...,b PKS Legislative Urges BPK to Audit Jabodebek...,0.000,0.081,0.919,-0.1280
1,1,Thu Nov 02 06:59:57 +0000 2023,@pakmul63 @PKSejahtera @FPKSDPRRI Udah dapet g...,0,0,0,0,in,frizkiramadhani,Udah dapet gambaran majunya industri kita sete...,b Have you got an idea of the progress of our ...,0.201,0.000,0.799,0.7003
2,2,Thu Nov 02 06:59:31 +0000 2023,@shortsidestory lrt jabodebek kan gak pke masi...,0,0,0,0,in,aliyaff28,lrt jabodebek kan gak pke masinis ya tapi yg j...,b Jabodebek LRT doesn t use a driver but the o...,0.195,0.107,0.698,0.5267
3,3,Thu Nov 02 06:59:21 +0000 2023,LRT Jabodebek kembali ke headway 15 menit seka...,1,0,1,2,in,jalur5_,LRT Jabodebek kembali ke headway 15 menit seka...,b LRT Jabodebek returns to headway once every ...,0.000,0.000,1.000,0.0000
4,4,Thu Nov 02 06:50:44 +0000 2023,Oh jadi ini ya alasannya LRT Jabodebek rolling...,0,0,0,1,in,Widoputranto,Oh jadi ini ya alasannya LRT Jabodebek rolling...,b Oh so this is the reason why the Jabodebek L...,0.000,0.124,0.876,-0.3400


In [ ]:
score = data['Compound'].values
sentiment = []
for i in score:
  if i >= 0.05 :
    sentiment.append("Positif")
  elif i <= -0.05 :
    sentiment.append("Negatif")
  else:
    sentiment.append("Netral")

data["Sentimen"] = sentiment
data.head()
del(data['Negative'])
del(data['label'])
del(data['Positive'])
del(data['Neutral'])
data.head()

,Unnamed: 0,created_at,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,username,clean,Compound,Sentimen
0,0,Thu Nov 02 07:06:28 +0000 2023,Legislatif PKS Desak BPK Audit Proyek LRT Jabo...,0,0,1,3,in,tvOneNews,Legislatif PKS Desak BPK Audit Proyek LRT Jabo...,-0.1280,Negatif
1,1,Thu Nov 02 06:59:57 +0000 2023,@pakmul63 @PKSejahtera @FPKSDPRRI Udah dapet g...,0,0,0,0,in,frizkiramadhani,Udah dapet gambaran majunya industri kita sete...,0.7003,Positif
2,2,Thu Nov 02 06:59:31 +0000 2023,@shortsidestory lrt jabodebek kan gak pke masi...,0,0,0,0,in,aliyaff28,lrt jabodebek kan gak pke masinis ya tapi yg j...,0.5267,Positif
3,3,Thu Nov 02 06:59:21 +0000 2023,LRT Jabodebek kembali ke headway 15 menit seka...,1,0,1,2,in,jalur5_,LRT Jabodebek kembali ke headway 15 menit seka...,0.0000,Netral
4,4,Thu Nov 02 06:50:44 +0000 2023,Oh jadi ini ya alasannya LRT Jabodebek rolling...,0,0,0,1,in,Widoputranto,Oh jadi ini ya alasannya LRT Jabodebek rolling...,-0.3400,Negatif


In [ ]:
data.to_csv('hasil_label_2.csv')